In [ ]:
import pandas as pd
import json
import re

In [ ]:
filename = input("File you want processed: ")
df = pd.read_csv((filename+'.csv'), index_col=0)
src_dst = df[["Source","Destination"]].copy()
src_dst.rename(columns={"Source":"source","Destination":"target"}, inplace=True)

In [ ]:
def ip_matcher(address):
    # Used to validate if string is an ipaddress
    ip = re.match(
        '^(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$', address)
    if ip:
        return True
    else:
        return False

src_dst['valid_src'] = src_dst.source.apply(ip_matcher)
src_dst['valid_target'] = src_dst.target.apply(ip_matcher)

valid_src_dest = src_dst[(src_dst.valid_src==True) & (src_dst.valid_target==True)]

In [ ]:
grouped_src_dst2 = valid_src_dest.groupby(["source","target"]).size().reset_index()

In [ ]:
grouped_src_dst = src_dst.groupby(["source","target"]).size().reset_index()

In [ ]:
unique_ips = pd.Index(grouped_src_dst['source']
                      .append(grouped_src_dst['target'])
                      .reset_index(drop=True).unique())

In [ ]:
group_dict = {}
counter = 0
for ip in unique_ips:
    breakout_ip = re.match("^(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})$", ip)
    if breakout_ip:
        net_id = '.'.join(breakout_ip.group(1,2,3,4))
        if net_id not in group_dict:
            counter += 1
            group_dict[net_id] = counter
        else:
            pass

In [ ]:
grouped_src_dst.rename(columns={0:'count'}, inplace=True)
temp_links_list = list(grouped_src_dst.apply(lambda row: {"source": row['source'], "target": row['target'], "value": row['count']}, axis=1))

In [ ]:
nodes_list = []
for ip in unique_ips:
    breakout_ip = re.match("^(\d{1,3})\.(\d{1,3})\.(\d{1,3})\.(\d{1,3})$", ip)
    if breakout_ip:
        net_id = '.'.join(breakout_ip.group(1,2,3,4))
        nodes_list.append({"id": str(group_dict.get(net_id)), "name":ip})

In [ ]:
links_list = []
for link in temp_links_list:
    linkVal = link['value']
    source = link['source']
    target = link['target']
    for nodez in nodes_list:
        if source == nodez['name']:
            new_source = nodez['id']
        if target == nodez['name']:
            new_target = nodez['id']
            record = {"value":linkVal, "source":new_source, "target": new_target}
            links_list.append(record)

In [ ]:
json_prep = {"nodes":nodes_list, "links":links_list}

In [ ]:
json_dump = json.dumps(json_prep, indent=1, sort_keys=True)

In [ ]:
filename_out = (filename + '.json')
json_out = open(filename_out,'w')
json_out.write(json_dump)
json_out.close()